In [1]:
!pip install langchain-google-genai langchain-core langgraph pydantic --quiet

In [2]:
!pip install -q google-generativeai


In [3]:
import os
from typing import Annotated, TypedDict, Union, Literal
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field

In [4]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["EMAIL_API_KEY"])

for m in genai.list_models():
    print(m.name)

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025

In [5]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client

In [6]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

Gmail authentication

In [7]:
GMAIL_SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
def authorize_gmail():
    creds = None

    # Check if token already exists
    if os.path.isfile('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', GMAIL_SCOPES
        )

    # If no valid credentials, start manual OAuth
    if not creds or not creds.valid:
        oauth_flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json',
            scopes=GMAIL_SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob'
        )

        auth_link, _ = oauth_flow.authorization_url(
            access_type='offline',
            include_granted_scopes='true'
        )

        print("\n OPEN THIS LINK IN YOUR BROWSER:\n")
        print(auth_link)

        auth_code = input("\n Enter the authorization code here: ").strip()

        oauth_flow.fetch_token(code=auth_code)
        creds = oauth_flow.credentials

        # Save token
        with open('token.json', 'w') as f:
            f.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)


In [8]:
gmail_service = authorize_gmail()
print(" Gmail authorization successful")


 Gmail authorization successful


In [21]:
import re

profile = gmail_service.users().getProfile(userId='me').execute()
my_email = profile['emailAddress']

name_part = my_email.split("@")[0].split(".")[0]
MY_NAME = re.sub(r'\d+', '', name_part).title()

print("Detected name:", MY_NAME)


Detected name: Harshithababu


Testing access

In [9]:
labels_response = gmail_service.users().labels().list(userId='me').execute()

print("\n Gmail Labels:")
for lbl in labels_response['labels']:
    print(lbl['name'])



 Gmail Labels:
CHAT
SENT
INBOX
IMPORTANT
TRASH
DRAFT
SPAM
CATEGORY_FORUMS
CATEGORY_UPDATES
CATEGORY_PERSONAL
CATEGORY_PROMOTIONS
CATEGORY_SOCIAL
YELLOW_STAR
STARRED
UNREAD
[Imap]/Drafts


In [10]:
import base64

def get_email_content(msg_id):
    msg = gmail_service.users().messages().get(
        userId='me',
        id=msg_id,
        format='full'
    ).execute()

    headers = msg['payload']['headers']
    subject, sender = "", ""

    for h in headers:
        if h['name'] == 'Subject':
            subject = h['value']
        if h['name'] == 'From':
            sender = h['value']

    body = ""
    parts = msg['payload'].get('parts', [])
    if parts:
        for part in parts:
            if part['mimeType'] == 'text/plain' and 'data' in part['body']:
                body = base64.urlsafe_b64decode(
                    part['body']['data']
                ).decode('utf-8')
    else:
        if 'data' in msg['payload']['body']:
            body = base64.urlsafe_b64decode(
                msg['payload']['body']['data']
            ).decode('utf-8')

    return subject, sender, body


In [23]:
count_results = gmail_service.users().messages().list(
    userId='me',
    labelIds=['UNREAD']
).execute()

total_unread = count_results.get('resultSizeEstimate', 0)
print("Total unread emails:", total_unread)


Total unread emails: 201


Checking unread emails and categorizing them

In [11]:
results = gmail_service.users().messages().list(
    userId='me',
    labelIds=['UNREAD'],
    maxResults=5
).execute()

unread_msgs = results.get('messages', [])

print("Top 5 unread emails found:\n")

def categorize_email(subject, body):
    text = (subject + " " + body).lower()

    if any(word in text for word in ["invoice", "payment", "due", "urgent"]):
        return "Important"
    elif any(word in text for word in ["facebook", "instagram", "twitter", "linkedin"]):
        return "Social"
    elif any(word in text for word in ["sale", "offer", "discount", "deal"]):
        return "Promotions"
    elif any(word in text for word in ["update", "newsletter", "alert", "notification"]):
        return "Updates"
    elif any(word in text for word in ["meeting", "project", "deadline", "assignment"]):
        return "Work"
    else:
        return "General"


for msg in unread_msgs:
    subject, sender, body = get_email_content(msg['id'])
    category = categorize_email(subject, body)

    print("From:", sender)
    print("Subject:", subject)
    print("Category:", category)
    print("-" * 40)


Top 5 unread emails found:

From: Harshitha B <harshithababu2316@gmail.com>
Subject: Attaching the documents needed for exam
Category: General
----------------------------------------
From: Google Play <googleplay-noreply@google.com>
Subject: Updates to how privacy settings work on Play
Category: Updates
----------------------------------------
From: NPTEL <onlinecourses@nptel.iitm.ac.in>
Subject: NPTEL Newsletter: Jan 2026: Vol 7: Week 4!
Category: Social
----------------------------------------
From: Adobe Acrobat Studio <mail@mail.adobe.com>
Subject: Generate a presentation from files in minutes
Category: General
----------------------------------------
From: LinkedIn <updates-noreply@linkedin.com>
Subject: Sruthi Nirmala and others share their thoughts on LinkedIn
Category: Social
----------------------------------------


In [12]:
!pip install google-generativeai


In [13]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)


models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-p

Gemini authentication

In [19]:
import os
import google.generativeai as genai

os.environ["GEMINI_API_KEY"] = ""
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

model = genai.GenerativeModel("gemini-2.5-flash")


Drafting response

In [22]:
def extract_name(sender):
    if "<" in sender:
        return sender.split("<")[0].strip()
    return sender.split("@")[0]


#CHECK FOR NO-REPLY EMAIL
def is_no_reply(sender):
    sender_lower = sender.lower()
    blocked = ["no-reply", "noreply", "do-not-reply", "donotreply"]
    return any(word in sender_lower for word in blocked)


#DRAFT REPLY USING GEMINI
def draft_reply(email_body, sender):
    name = extract_name(sender)

    if not email_body.strip():
        return f"Hello {name},\n\nThank you for your email. I will get back to you shortly.\n\nBest regards."

    prompt = f"""
Write a polite and professional email reply.

Greeting Rules:
- If the email sounds formal (job, official, request, professor, company), start with: Dear {name},
- If the tone is neutral, start with: Hello {name},
- If the tone is friendly or casual, start with: Hi {name},

Other Rules:
- Do not use placeholders
- Keep the reply clear and professional
Reply to this email:

{email_body}
"""

    response = model.generate_content(prompt)
    return response.text


#SEND EMAIL FUNCTION
def send_email(to_email, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to_email
    message['subject'] = "Re: " + subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    gmail_service.users().messages().send(
        userId='me',
        body={'raw': raw}
    ).execute()


# MAIN LOOP
for msg in unread_msgs:
    subject, sender, body = get_email_content(msg['id'])

    print("=" * 60)
    print("From:", sender)
    print("Subject:", subject)

    #  Skip no-reply emails
    if is_no_reply(sender):
        print("No-reply email skipped")
        continue

    reply = draft_reply(body, sender)

    print("\nDrafted Reply:\n")
    print(reply)

    while True:
        choice = input("\nSend email? (yes / no / edit): ").strip().lower()

        if choice == "yes":
            send_email(sender, subject, reply)
            print("Email sent")
            break

        elif choice == "edit":
            edited_reply = input("\nEnter your edited reply:\n")
            send_email(sender, subject, edited_reply)
            print("Edited email sent")
            break

        elif choice == "no":
            print("Email skipped")
            break

        else:
            print("Please type 'yes', 'no', or 'edit'.")


From: Harshitha B <harshithababu2316@gmail.com>
Subject: Attaching the documents needed for exam

Drafted Reply:

Dear Harshitha B,

Thank you for sending the necessary documents for my exams. I have received them and will make sure to review them thoroughly. I appreciate your kind wishes.

Regards,
Harshitha

Send email? (yes / no / edit): yes
Email sent
From: Google Play <googleplay-noreply@google.com>
Subject: Updates to how privacy settings work on Play
No-reply email skipped
From: NPTEL <onlinecourses@nptel.iitm.ac.in>
Subject: NPTEL Newsletter: Jan 2026: Vol 7: Week 4!

Drafted Reply:

Hello NPTEL,

Thank you for this detailed update. I appreciate receiving the information regarding exam registrations, course enrollment extensions, and the interesting list of NPTEL+ workshops. This is very useful.

Regards,

Send email? (yes / no / edit): no
Email skipped
From: Adobe Acrobat Studio <mail@mail.adobe.com>
Subject: Generate a presentation from files in minutes

Drafted Reply:

Hello